# 実践コンピュータビジョン

[実践 コンピュータビジョン](https://www.oreilly.co.jp/books/9784873116075/)  
[Multiple View Geometry in Computer Vision](https://www.amazon.co.jp/dp/0521540518)(以下、MVGCVと呼ぶ) 

演習ノート

# 3. 画像間の写像

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pathlib import Path    
import sys
sys.path.append("../")
from impro import impro,detection,homography

## 3.1 ホモグラフィー

>ホモグラフィーは、ある平面から別の平面へ写像する2Dの射影変換です。

### 同次座標
2次元座標上の点は$(x,y)$と2つの数の組で表せるが、もう1つ座標を加えて、$(x,y,1)$として表す。このような座標のことをを同次座標(homogeneous coordinate)と呼ぶ

### アフィン変換
座標$(x,y)$の点を座標変換して、さらに$(t_x,t_y)$平行移動する場合を考える。
$$
{\begin{cases}
x' = a_1x+a_2y+t_x \\
y' = a_3x+a_4y+t_y
\end{cases}\tag{2}
}
$$

同次座標を用いればこれを行列の積で表すことができる。
$$
\begin{pmatrix}
x' \\
y' \\
1
\end{pmatrix}=
\begin{pmatrix}
a_1 & a_2 & t_x \\
a_3 & a_4 & t_y \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1
\end{pmatrix}
$$

同次座標のメリットはほかにもいろいろある。同次座標、射影平面、アフィン変換、射影変換について詳しくは以下を参照  
- [完全に理解するアフィン変換 - Qiita](https://qiita.com/koshian2/items/c133e2e10c261b8646bf)  
- [Python, OpenCVで幾何変換（アフィン変換・射影変換など） | note.nkmk.me](https://note.nkmk.me/python-opencv-warp-affine-perspective/)
- [射影平面をいくつかの方法で図示してみる - あおいろメモ](https://solkul.hatenablog.com/entry/2021/01/15/113702)  

### 回転
OpenCVでは、画像の左上を原点$(0,0)$とし、右方向を$x$プラス方向、下方向を$y$プラス方向に取る。数学のxy座標系とは異なるので注意。  
一方、`cv2.getRotationMatrix2D`は数学でのxy座標系と同様、反時計回りを正に取る。  
従って、数学では回転行列が
$$
\begin{pmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{pmatrix}
$$
であるのに対し、OpenCVでは
$$
\begin{pmatrix}
\cos\theta & \sin\theta \\
- \sin\theta & \cos\theta
\end{pmatrix}
$$
となる。
詳しくはドキュメント参照  
[Geometric Image Transformations — opencv v2.2 documentation](http://opencv.jp/opencv-2.2_org/py/imgproc_geometric_image_transformations.html#getrotationmatrix2d)

### 正規化と同次座標への変換

In [41]:
# 自作のimpro.homographyに3つの組の最後の座標で割り、正規化する関数を実装
points=np.arange(12).reshape(3,-1)
points

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [42]:
homography.normalize(points)

array([[0.        , 0.11111111, 0.2       , 0.27272727],
       [0.5       , 0.55555556, 0.6       , 0.63636364],
       [1.        , 1.        , 1.        , 1.        ]])

In [43]:
# 同次座標への変換
points=np.arange(12).reshape(2,-1)
points

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [61]:
homography.make_homog(points)

array([[ 0.,  1.,  2.,  3.,  4.,  5.],
       [ 6.,  7.,  8.,  9., 10., 11.],
       [ 1.,  1.,  1.,  1.,  1.,  1.]])

### 3.1.1 DLTアルゴリズム
射影変換はホモグラフィー行列$H$を使って次のように表すことができる
$$
\begin{pmatrix}
x' \\
y' \\
w'
\end{pmatrix}=
\begin{pmatrix}
h_1 & h_2 & h_3 \\
h_4 & h_5 & h_6 \\
h_7 & h_8 & h_9
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
w
\end{pmatrix}
$$
$$
\boldsymbol{x}'=H\boldsymbol{x}
$$

ある点$(x,y)$から$(x',y')$へ変換する$H$を求めるのがDirect Linear Transformation(DLT)アルゴリズムである。詳しい導出は[Multiple View Geometry in Computer Vision](https://www.amazon.co.jp/dp/0521540518)の4.1(p88)を参照。以下はその概要と、補足を記す。

DLTアルゴリズムは外積$\boldsymbol{x}'$同士の外積$\boldsymbol{x}'\times H\boldsymbol{x}$が0であることを用いて導出する。
$$
\boldsymbol{x}'_i\times H\boldsymbol{x}_i=
\begin{pmatrix}
y'_i\boldsymbol{h}^{3T}-w'_i\boldsymbol{h}^{2T} \\
w'_i\boldsymbol{h}^{1T}-x'_i\boldsymbol{h}^{3T} \\
x'_i\boldsymbol{h}^{2T}-y'_i\boldsymbol{h}^{1T}
\end{pmatrix}
=\boldsymbol{0}
$$
これを変形することで、$A\boldsymbol{h}=\boldsymbol{0}$という式になる。

#### 線形代数のIm:像、Ker:核との関係
MVGCVのp90では以下のように述べている。
>変換前、変換後の点の対応1つにつき、2つの独立な線形方程式が得られる。もしそのような対応が4点分あったら、8つの独立な線形方程式が得られる。そうすると、$A$のランクが8となるので、$\boldsymbol{h}$の解は非ゼロの1次元ベクトル空間になる。これによって$\boldsymbol{h}$はスケールの違いを除いて一意に決まる。


これについて捕捉する。$\boldsymbol{h}=\mathrm{Ker}A$(Aのカーネル)であるので、もともとの$h$がとりうるベクトル空間を$V$とすると
$$
\mathrm{dim}V=\mathrm{rank}A+\mathrm{dim}\boldsymbol{h}
$$
とできる。今、8つの独立な線形方程式が得られ、$\mathrm{rank}A=8$となり、$\mathrm{dim}V=9$なので$\mathrm{dim}\boldsymbol{h}=1$となる、つまり$\boldsymbol{h}$は行列の固有ベクトルのように大きさが任意の向きが決まったベクトルとなるということである。なお、$\mathrm{dim}V=\mathrm{rank}A+\mathrm{dim}\boldsymbol{h}$については以下を参照


[線形写像の次元定理dim V = rank f + dim ker fの証明 | 数学の景色](https://mathlandscape.com/rank-ker-dim/)

#### 特異値分解 singular value decomposition(SVD)との関係
一般的に$h$を求めるには$A^TA$の最小の固有値に対応する固有ベクトルが$h$となる。具体的には$A$を特異値分解し、
$$
A=U\Sigma V^T
$$
この右特異ベクトル$V^T$の8行目のベクトルが最適な$h$となる。  
この導出に利用する線形方程式の解法についてはMVGCVのA.5(p588)や日本語の場合次の資料を参照。  
- [【解説】 一般逆行列](https://www.slideshare.net/wosugi/ss-79624897)
- [特異値分解を詳しく解説 - Qiita](https://qiita.com/kidaufo/items/0f3da4ca4e19dc0e987e)

[特異値分解を詳しく解説 - Qiita](https://qiita.com/kidaufo/items/0f3da4ca4e19dc0e987e)で述べているように、numpyの`np.linalg.svd`場合、$m\times
n$でランクが$r<m,n$の行列$A$を特異値分解すると、次の図で示されるような分解のされ方をする。
![img](figures/01SVD_python.png)

DLTアルゴリズムでは$A$は$m\times9$なので、次の図のようになる。そして$V^T$の8行目が最適な$h$となる。(条件を満たすホモグラフィーが存在する4点以上の厳密な座標の対応が与えられたときには$A$はランク8、つまり$r=8$となるが、そうでない場合は$r\neq8$となりうる)
![img](figures/02SVD_DLT.png)

[実践 コンピュータビジョン](https://www.oreilly.co.jp/books/9784873116075/)のp57のサンプルコードでは次のように$V^T$の8行目を取得し、$H$としている。
```python
U,S,V = np.linalg.svd(mat_A)
H = V[8].reshape((3,3))
```

In [52]:
src=np.array(
    [
        [0,0],
        [0,1],
        [1,0],
        [1,1]
    ],
    np.float32).T
dest=src*10
dest

array([[ 0.,  0., 10., 10.],
       [ 0., 10.,  0., 10.]], dtype=float32)

In [63]:
src_homo=homography.make_homog(src)
dest_homo=homography.make_homog(dest)
dest_homo

array([[ 0.,  0., 10., 10.],
       [ 0., 10.,  0., 10.],
       [ 1.,  1.,  1.,  1.]])

In [67]:
m = np.mean(src_homo[:2], axis=1)
maxstd = np.max(np.std(src_homo[:2], axis=1)) + 1e-9

c_1:np.ndarray = np.diag([1/maxstd, 1/maxstd, 1])
c_1[0][2] = -m[0]/maxstd
c_1[1][2] = -m[1]/maxstd
src_cnv = c_1 @ src_homo

In [70]:
c_1

array([[ 2.,  0., -1.],
       [ 0.,  2., -1.],
       [ 0.,  0.,  1.]])

In [69]:
src_homo

array([[0., 0., 1., 1.],
       [0., 1., 0., 1.],
       [1., 1., 1., 1.]])

In [77]:
m = np.mean(src_homo[:2], axis=1)
maxstd = np.max(np.std(src_homo[:2], axis=1)) + 1e-9
c_1:np.ndarray = np.diag([1/maxstd, 1/maxstd, 1])
c_1[0][2] = -m[0]/maxstd
c_1[1][2] = -m[1]/maxstd
fp = c_1 @ src_homo
# 対応点
m = np.mean(dest_homo[:2], axis=1)
maxstd = np.max(np.std(dest_homo[:2], axis=1)) + 1e-9
c_2:np.ndarray = np.diag([1/maxstd, 1/maxstd, 1])
c_2[0][2] = -m[0]/maxstd
c_2[1][2] = -m[1]/maxstd
tp = c_2 @ dest_homo

In [79]:
nbr_correspondences = src_homo.shape[1]

In [81]:
mat_A = np.zeros((2*nbr_correspondences,9))
for i in range(nbr_correspondences):
    mat_A[2*i] = [-fp[0][i],-fp[1][i],-1,0,0,0,
        tp[0][i]*fp[0][i],tp[0][i]*fp[1][i],tp[0][i]]
    mat_A[2*i+1] = [0,0,0,-fp[0][i],-fp[1][i],-1,
        tp[1][i]*fp[0][i],tp[1][i]*fp[1][i],tp[1][i]]

In [83]:
U,S,V = np.linalg.svd(mat_A)

In [89]:
U.shape

(8, 8)

In [90]:
V.shape

(9, 9)